In [57]:
# 匯入所需的套件

# 引入 Selenium 的 webdriver 模組，用於操作瀏覽器 打開網頁、點擊元素、獲取內容等
from selenium import webdriver

# 引入 Service 類，用於設置 ChromeDriver 的服務 需要時啟動和停止
from selenium.webdriver.chrome.service import Service

# 引入 ChromeDriverManager 類，能自動下載和管理適合當前 Chrome 版本的 ChromeDriver
from webdriver_manager.chrome import ChromeDriverManager

# 引入異常類型，用於處理網頁元素查找和超時錯誤
# TimeoutException：當等待某個條件超過指定時間但條件未滿足時拋出的異常。
# NoSuchElementException：當嘗試查找不存在的網頁元素時拋出的異常。
from selenium.common.exceptions import TimeoutException, NoSuchElementException

# 引入 WebDriverWait 類，用於設置顯示等待 以確保某些條件滿足後再進行後續操作
from selenium.webdriver.support.ui import WebDriverWait

# 引入 expected_conditions 模組，用於設定等待條件
from selenium.webdriver.support import expected_conditions as EC

# 引入 By 類，用於指定元素的查找方式
from selenium.webdriver.common.by import By

# 引入 sleep 函數，用於添加延遲
from time import sleep

# 整理 json 使用的工具
import json

# 整理 csv 使用的工具
import csv

# 執行 command 的時候用的
import os

# 資料整理
import pandas as pd

In [58]:
# 瀏覽器設定
my_options = webdriver.ChromeOptions()
my_options.add_argument("--start-maximized")           #最大化視窗
my_options.add_argument("--incognito")                 #開啟無痕模式
my_options.add_argument("--disable-popup-blocking")    #禁用彈出攔截
my_options.add_argument("--disable-notifications")     #取消 chrome 推播通知
my_options.add_argument("--lang=zh-TW")                #設定為正體中文

# 開啟自動控制瀏覽器
driver = webdriver.Chrome(options = my_options,)

In [59]:
# 建立資料夾
folderPath = '591_data'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

# 放置爬取的資料
listData = []

In [60]:
# 爬蟲流程 function

# 走訪頁面
def visit():
    driver.get("https://market.591.com.tw/list?regionId=3&postType=2,8")
    
    # 等待一下
    sleep(1)

# 按鈕選擇器
def button():
    try:
        # 等待元素
        WebDriverWait(driver, 1).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, "button.guide-button")))
        
        # 使用 CSS 選擇器查找按鈕
        button = driver.find_element(By.CSS_SELECTOR, "button.guide-button").click()
    except NoSuchElementException:
        print("沒有彈跳式資訊")

    # 等待一下
    sleep(1)

# 篩選 (選項)
def filter():
    try:
        # 等待元素
        WebDriverWait(driver, 1).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, ".t5-dropdown-select")))
        
        # 選擇區域
        driver.find_element(By.CSS_SELECTOR, ".t5-dropdown-select").click()

        # 選擇永和
        driver.find_element(By.CSS_SELECTOR, "#areaFilterItemEle17").click()

        # 選擇確定
        driver.find_elements(By.CSS_SELECTOR, "section.grid-filter-btn button")[1].click()

    except NoSuchElementException:
        print("篩選出錯")

    # 等待一下
    sleep(1)

# 滾動頁面
def scroll():
    innerHeight = 0  # 瀏覽器內部的高度
    offset = 0       # 當前捲動的量(高度)
    count = 0        # 累計無效滾動次數
    limit = 3        # 最大無效滾動次數

    # 在捲動到沒有元素動態產生前，持續捲動
    while count <= limit:
        # 每次移動高度
        offset = driver.execute_script('return document.documentElement.scrollHeight;')

        # 捲軸往下滑動
        driver.execute_script(f'''
            window.scrollTo({{
                top:{offset},
                behavior:'smooth'
            }});
        ''')

        # 強制等待，此時若有新元素生成，瀏覽器內部高度會自動增加
        sleep(3)
        
        # 透過執行 js 語法來取得捲動後的當前總高度
        innerHeight = driver.execute_script('return document.documentElement.scrollHeight;')
        
        # 經過計算，如果滾動距離(offset)大於等於視窗內部總高度(innerHeight)，代表已經到底了
        if offset == innerHeight:
            count += 1

        # 測試用，捲動超過一定的距離，就結束程式
        # if offset >= 600:
        #     break

# 爬取資料
def scrape_data():
    # 使用全域變數
    global listData
    
    # 清空存放資料的變數
    listData.clear()

    # 取得主要元素的集合
    elements = driver.find_elements(By.CSS_SELECTOR,'a.community-card')

    for card in elements:
        print("=" * 30)

        name = card.find_element(By.CSS_SELECTOR, 'h3 em[data-v-c7d6a2ce]').get_attribute('innerText')

        try:
            address = card.find_elements(By.CSS_SELECTOR, 'p em[data-v-c7d6a2ce]')[1].get_attribute('innerText')
        except:
            address = card.find_elements(By.CSS_SELECTOR, 'p em[data-v-c7d6a2ce]')[-1].get_attribute('innerText')

        try:
            price = card.find_element(By.CSS_SELECTOR, 'span.price-info').get_attribute('innerText')
        except NoSuchElementException:
            price = "未提供價格"

        link = card.get_attribute('href')
        
        print(name)
        print(address)
        print(price)
        print(link)

        listData.append({
            "名稱": name,
            "地址":address,
            "價格":price,
            "網址":link
        })

# 將 list 存成 json
def saveJSON():
    with open(f"{folderPath}/591_data.json", "w", encoding='utf-8') as file:
        file.write( json.dumps(listData, ensure_ascii=False, indent=4) )


# 將 list 存成 csv
def saveCSV():
    # 讀取 JSON 資料
    with open(f"{folderPath}/591_data.json", "r", encoding='utf-8') as file:
        data = json.load(file)

    # 將 JSON 資料轉換為 DataFrame
    df = pd.DataFrame(data)
    
    # 儲存為 CSV 文件
    df.to_csv(f"{folderPath}/591_data.csv", index=False, encoding='utf-8-sig')

In [62]:
if __name__ == '__main__':
    visit()
    button()
    filter()
    scroll()
    scrape_data()
    saveJSON()
    saveCSV()

樺福千金
永和區永和路二段168號
82.0萬/坪
https://market.591.com.tw/11070
漢寶臺大苑
永和區成功路一段132號之1
63.8萬/坪
https://market.591.com.tw/5912088
漢皇盛世
永和區永和路一段70號旁
61.5萬/坪
https://market.591.com.tw/49547
東家創世紀大廈
永和區中山路一段236號
68.4萬/坪
https://market.591.com.tw/28708
中一秀彥
永和區中正路617號
57.9萬/坪
https://market.591.com.tw/10995
雙和百順客
永和區中和路393號
67.9萬/坪
https://market.591.com.tw/10936
新時代
永和區中山路一段38號
72.2萬/坪
https://market.591.com.tw/25888
百鴻希爾頓
永和區福和路353號
52.0萬/坪
https://market.591.com.tw/28703
北城玫塊
永和區民生路46巷10號
61.8萬/坪
https://market.591.com.tw/10462
天琴大廈
永和區環河東路四段2號
57.9萬/坪
https://market.591.com.tw/28701
凱旋臻品
永和區永貞路360巷1號
66.3萬/坪
https://market.591.com.tw/25786
頂溪大苑
永和區中山路一段28巷口旁
90.5萬/坪
https://market.591.com.tw/5935126
仁愛國寶
永和區仁愛路306巷71號
72.0萬/坪
https://market.591.com.tw/11044
銀河水都
永和區成功路二段227號
66.8萬/坪
https://market.591.com.tw/28732
長隄
永和區環河西路二段209巷11號
59.5萬/坪
https://market.591.com.tw/10960
金歡喜商業大廈
永和區中和路489號
61.9萬/坪
https://market.591.com.tw/10978
勝開大地
永和區保安路236號
74.1萬/坪
https://market.591.com.tw/44487
尚林苑
永和區安

In [63]:
# 關閉瀏覽器
driver.quit()